## Ensemble

ex) 무작위로 선택된 수천 명의 사람에게 복잡한 질문을 하고 대답을 모은다고 가정했을때, 이렇게 모은 답이 전문가의 답보다 낫다.<br>
이를 **대중의 지혜**라고 한다.

이와 비슷하게 일련의 예측기(분류나 회귀 모델)로부터 예측을 수집하면 가장 좋은 모델 하나보다 더 좋은 예측을 얻을 수 있을 것이다.<br>
이러한 일련의 예측기를 **앙상블**이라고 부르기 때문에 이를 **앙상블 학습(ensemble learning)**이라고 하며, 앙상블 학습 알고리즘을 **앙상블 방법(ensemble method)**라고 한다.




### VotingClassifier

- 투표 기반 분류기

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings('ignore')
from IPython.display import Image

In [2]:
from sklearn.datasets import make_moons
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import VotingClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC

In [3]:
# 데이터셋 불러오기
datasets = make_moons(n_samples = 1000)
data = datasets[0]
target = datasets[1]

In [4]:
from sklearn.model_selection import train_test_split

X_train,X_test,y_train,y_test = train_test_split(data, target, test_size = 0.2, random_state = 42)

In [5]:
log_clf = LogisticRegression()
svm_clf = SVC()
rnd_clf = RandomForestClassifier()

### Hard Voting

- Classifier의 결과들을 집계하여 다수의 표를 얻은 클래스를 최종 예측값으로 채택 

### Soft Voting

- 각 분류기의 예측을 평균내어 확률이 가장 높은 클래스의 예측값을 채택

In [6]:
# 분류기를 만들어 VotingClassifier 훈련
vot_clf = VotingClassifier(estimators = [('lr',log_clf),('svc',svm_clf),('rnd',rnd_clf)], voting = 'hard')

vot_clf.fit(X_train,y_train)

VotingClassifier(estimators=[('lr', LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='warn',
          n_jobs=None, penalty='l2', random_state=None, solver='warn',
          tol=0.0001, verbose=0, warm_start=False)), ('svc', SVC(C=1...obs=None,
            oob_score=False, random_state=None, verbose=0,
            warm_start=False))],
         flatten_transform=None, n_jobs=None, voting='hard', weights=None)

In [7]:
from sklearn.metrics import accuracy_score

In [8]:
for clf in (log_clf,svm_clf,rnd_clf,vot_clf) :
    clf.fit(X_train,y_train)
    y_pred = clf.predict(X_test)
    print(clf.__class__.__name__, accuracy_score(y_test, y_pred))

LogisticRegression 0.89
SVC 1.0
RandomForestClassifier 0.995
VotingClassifier 1.0


#### 투표 기반 분류기가 다른 개별 분류기보다 성능이 조금 더 높다.

모든 분류기가 클래스의 확률을 예측할 수 있다면 계별 분류기의 예측을 평균 내어 확률이 가장 높은 클래스를 예측할 수 있다.<br>
-> **soft voting** 

다양한 분류기를 만드는 방법중 하나인 **Voting**분류는 각기 다른 훈련 알고리즘을 사용하는 것이었다.

다른 방법은 같은 알고리즘을 사용하고 훈련 세트의 서브셋을 무작위로 구성하여 분류기를 각기 다르게 학습시킨다.

## Bagging, Pasting


### Bagging : 훈련 세트에서 중복을 허용하여 샘플링하는 방식

### Pasting : 중복을 허용하지 않고 샘플링하는 방식


In [9]:
datasets = make_moons(n_samples = 500)
data = datasets[0]
target = datasets[1]

from sklearn.model_selection import train_test_split

X_train,X_test,y_train,y_test = train_test_split(data, target, test_size = 0.2, random_state = 42)

In [10]:
# decision tree 500개의 앙상블을 훈련시키기 - 각 분류기는 훈련 세트에서 중복을 허용하여 무작위로 선택된 100개의 샘플로 훈련
from sklearn.ensemble import BaggingClassifier
from sklearn.tree import DecisionTreeClassifier

bag_clf = BaggingClassifier(
DecisionTreeClassifier(), n_estimators = 500, max_samples = 100, bootstrap = True, n_jobs = -1)

bag_clf.fit(X_train,y_train)
y_pred = bag_clf.predict(X_test)
accuracy_score(y_test,y_pred)

1.0

### OOB(Out Of Bag)평가

- 배깅은 중복을 허용하기에 어떤 샘플은 여러 번 샘플링되고 어떤 것은 전혀 선택되지 않을 수 있다.

- bootstrap이 True일 경우 훈련 세트의 크기만큼인 m개 샘플을 선택한다 -> 이는 평균적으로 각 예측기에 훈련 샘플의 63%정도만 샘플링 됨을 의미함.

***
> n개의 관측치가 존재하는 데이터에서 N개의 표본을 단순확률 반복추출하게 될 경우 확률.<br>
$($1 - $ $$1\over n$$)^n$

> 이 때 N이 충분히 클 경우
$($1 - $ $$1\over e$$)^n = 0.63212$

훈련 샘플의 63%만 샘플링 될 경우, 나머지 37%를 **OOB** 샘플이라고 부른다.

In [11]:
# OOB 평가 수행
bag_clf = BaggingClassifier(
DecisionTreeClassifier(), n_estimators = 500, bootstrap = True, oob_score = True, random_state = 42)

bag_clf.fit(X_train,y_train)
# OOB평가 점수 결과
bag_clf.oob_score_

0.995

해당 BaggingClassifier는 테스트 셋에서 약 99.5%의 정확도를 얻을것으로 보임.

In [12]:
from sklearn.metrics import accuracy_score
y_pred = bag_clf.predict(X_test)
accuracy_score(y_test, y_pred)

1.0

테스트 세트에서도 99.5%의 정확도를 얻었다.

In [13]:
# 각 훈련 샘플의 클래스 확률
bag_clf.oob_decision_function_

# 1열 : 음성 클래스 확률
# 2열 : 양성 클래스 확률

array([[0.        , 1.        ],
       [0.08917197, 0.91082803],
       [1.        , 0.        ],
       [1.        , 0.        ],
       [0.        , 1.        ],
       [0.        , 1.        ],
       [1.        , 0.        ],
       [0.        , 1.        ],
       [1.        , 0.        ],
       [1.        , 0.        ],
       [1.        , 0.        ],
       [0.        , 1.        ],
       [0.        , 1.        ],
       [0.        , 1.        ],
       [0.99489796, 0.00510204],
       [1.        , 0.        ],
       [1.        , 0.        ],
       [0.        , 1.        ],
       [1.        , 0.        ],
       [0.        , 1.        ],
       [1.        , 0.        ],
       [1.        , 0.        ],
       [0.        , 1.        ],
       [0.        , 1.        ],
       [0.        , 1.        ],
       [1.        , 0.        ],
       [0.        , 1.        ],
       [1.        , 0.        ],
       [1.        , 0.        ],
       [0.        , 1.        ],
       [0.

## 랜덤포레스트

- 일반적으로 DecisionTree에 배깅이나 페이스팅을 적용한 결정트리의 앙상블이다.

- **Max_samles**를 훈련 세트의 크기로 지정한다. (default는 훈련 세트 전체)

In [14]:
# random_forest 모델 
from sklearn.ensemble import RandomForestClassifier

# 모델 훈련 16개의 leaf노드를 갖는 500개 트리로 구성된 랜덤 포레스트
rnd_clf = RandomForestClassifier(n_estimators = 500, max_leaf_nodes = 16, n_jobs = -1)
# 모델 적합
rnd_clf.fit(X_train,y_train)
y_pred_rf = rnd_clf.predict(X_test)

## 특성 중요도

랜덤 포레스트는 특성의 상대적 중요도를 측정하기가 쉽다.

**feature_importance**를 사용하여 계산하며 중요도의 전체 합이 1이 되도록 결과값을 정규화한다.

In [15]:
from sklearn.datasets import load_iris
iris = load_iris()
# 500개의 트리로 구성된 iris의 랜덤 포레스트
rnd_clf = RandomForestClassifier(n_estimators = 500, n_jobs = -1)

# 모델 적합
rnd_clf.fit(iris['data'],iris['target'])

# 변수별 중요도 표시
for name, score in zip(iris['feature_names'], rnd_clf.feature_importances_) :
    print(name, score)

sepal length (cm) 0.09846658699513185
sepal width (cm) 0.026100635990750524
petal length (cm) 0.4388537182153215
petal width (cm) 0.43657905879879616


## 부스팅

- 약한 학습기를 여러개 연결하여 강한 학습기를 만드는 앙상블 방법

- 부스팅 방법 중 가장 인기 있는 방법은 **AdaBoost**와 **gradient boosting**이다.

### AdaBoosting

- 이전 예측기를 보완하기 위한 방법중 하나는 과소적합했던 훈련 샘플의 가중치를 더 높여 학습하기 어려운 샘플에 점점 더 맞추는 방법

#### 에이다부스트 분류기를 만들 때 
***
먼저 알고리즘이 기반이 되는 첫번째 분류기를 훈련 세트에서 훈련시키고 예측을 만든다.

그다음에 알고리즘이 잘못 분류된 훈련 샘플의 가중치를 상대적으로 높인다.

두 번째 분류기는 업데이트된 가중치를 사용해 훈련 세트에서 훈련하고 다시 예측을 만든다.

그다음에 다시 가중치를 업데이트한다.

***
![screensh](./images/ensemble/adaboost.png)


In [16]:
from sklearn.datasets import load_iris

iris = load_iris()
data = iris['data']
target = iris['target']

from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test = train_test_split(data,target, test_size = 0.2, random_state = 42)

In [17]:
# adaboost 모델 학습
# 트리 노드 1개
# 앙상블 200개
# 가중치 계산 알고리즘 "SAMME.R"
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import AdaBoostClassifier
ada_clf = AdaBoostClassifier(
DecisionTreeClassifier(max_depth = 1), n_estimators = 200, 
                      algorithm = "SAMME.R", learning_rate = 0.5)

#### adaboost 파라미터

**n_estimators** = 생성할 약한 학습기의 갯수를 지정(default = 50)<br>
**learning_rate** = 학습을 진행할 때마다 적용하는 학습률(0 ~ 1)


n_estimators를 늘린다면 학습기의 결정 경계가 늘어나면서 모델이 복잡해진다.

learning_rate를 줄인다면 가중치 갱신의 변동폭이 감소해서 여러 학습기 결정경계의 차이가 줄어든다

In [18]:
ada_clf.fit(X_train,y_train)

AdaBoostClassifier(algorithm='SAMME.R',
          base_estimator=DecisionTreeClassifier(class_weight=None, criterion='gini', max_depth=1,
            max_features=None, max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, presort=False, random_state=None,
            splitter='best'),
          learning_rate=0.5, n_estimators=200, random_state=None)

### Gradient Boosting

앙상블에 이전까지의 오차를 보정하도록 예측기를 순차적으로 추가한다는 과정은 adaboost와 동일하다.

하지만 adaboost처럼 샘플의 가중치를 수정하는 대신 이전 예측기가 만든 **잔여 오차**에 새로운 예측기를 학습시킨다.


In [19]:
# 2차 곡선 train set 생성
import numpy as np
np.random.seed(42)
X = np.random.rand(100,1) -0.5
y = 2*(X**2) + np.random.rand(100)

print(X,y)

[[-0.12545988]
 [ 0.45071431]
 [ 0.23199394]
 [ 0.09865848]
 [-0.34398136]
 [-0.34400548]
 [-0.44191639]
 [ 0.36617615]
 [ 0.10111501]
 [ 0.20807258]
 [-0.47941551]
 [ 0.46990985]
 [ 0.33244264]
 [-0.28766089]
 [-0.31817503]
 [-0.31659549]
 [-0.19575776]
 [ 0.02475643]
 [-0.06805498]
 [-0.20877086]
 [ 0.11185289]
 [-0.36050614]
 [-0.20785535]
 [-0.13363816]
 [-0.04393002]
 [ 0.28517596]
 [-0.30032622]
 [ 0.01423444]
 [ 0.09241457]
 [-0.45354959]
 [ 0.10754485]
 [-0.32947588]
 [-0.43494841]
 [ 0.44888554]
 [ 0.46563203]
 [ 0.30839735]
 [-0.19538623]
 [-0.40232789]
 [ 0.18423303]
 [-0.05984751]
 [-0.37796177]
 [-0.00482309]
 [-0.46561148]
 [ 0.4093204 ]
 [-0.24122002]
 [ 0.16252228]
 [-0.18828892]
 [ 0.02006802]
 [ 0.04671028]
 [-0.31514554]
 [ 0.46958463]
 [ 0.27513282]
 [ 0.43949894]
 [ 0.39482735]
 [ 0.09789998]
 [ 0.42187424]
 [-0.4115075 ]
 [-0.30401714]
 [-0.45477271]
 [-0.17466967]
 [-0.11132271]
 [-0.22865097]
 [ 0.32873751]
 [-0.14324667]
 [-0.21906549]
 [ 0.04269608]
 [-0.35907

In [20]:
# 데이터셋으로 첫 번째 DecisionTreeRegressor 생성
from sklearn.tree import DecisionTreeRegressor

tree_reg1 = DecisionTreeRegressor(max_depth = 2)
tree_reg1.fit(X,y)

DecisionTreeRegressor(criterion='mse', max_depth=2, max_features=None,
           max_leaf_nodes=None, min_impurity_decrease=0.0,
           min_impurity_split=None, min_samples_leaf=1,
           min_samples_split=2, min_weight_fraction_leaf=0.0,
           presort=False, random_state=None, splitter='best')

In [21]:
# 첫 번째 예측기에서 생긴 잔여 오차에 두번째 DecisionTreeRegressor 생성
y2 = y - tree_reg1.predict(X)
tree_reg2 = DecisionTreeRegressor(max_depth = 2)
tree_reg2.fit(X,y2)

DecisionTreeRegressor(criterion='mse', max_depth=2, max_features=None,
           max_leaf_nodes=None, min_impurity_decrease=0.0,
           min_impurity_split=None, min_samples_leaf=1,
           min_samples_split=2, min_weight_fraction_leaf=0.0,
           presort=False, random_state=None, splitter='best')

In [22]:
# 2 번째 예측기가 만든 잔여 오자에 세 번재 회귀 모델 훈련 
y3 = y2 - tree_reg2.predict(X)
tree_reg3  = DecisionTreeRegressor(max_depth = 2)
tree_reg3.fit(X,y3)

DecisionTreeRegressor(criterion='mse', max_depth=2, max_features=None,
           max_leaf_nodes=None, min_impurity_decrease=0.0,
           min_impurity_split=None, min_samples_leaf=1,
           min_samples_split=2, min_weight_fraction_leaf=0.0,
           presort=False, random_state=None, splitter='best')

In [23]:
X_new = np.array([[0.8]])

In [24]:
# 새로운 샘플에 대한 예측을 만드려면 모든 트리의 예측을 더한다.
y_pred = sum(tree.predict(X_new) for tree in (tree_reg1, tree_reg2, tree_reg3))

## xgboost

**Gradient Boosting**의 단점인 느린 시간, 과적합 규제 등을 해결한 알고리즘이다.

#### xgboost 장점
***
1) 뛰어난 성능<br>
2) GBM대신 빠른 수행 시간<br>
3) 과적합 규제<br>
4) 트리 가지치기 : 긍정 이득이 없는 분할을 가지치기해서 분할을 줄임<br>
5) 자체 내장된 교차 검증<br>
6) 결손값 자체 처리

***


In [82]:
from sklearn.datasets import load_breast_cancer
from sklearn.model_selection import train_test_split

In [83]:
# 데이터 불러오기
br_ca = load_breast_cancer()

In [84]:
data = br_ca.data
target = br_ca.target

In [85]:
cancer_df = pd.DataFrame(data, columns = br_ca['feature_names'])
cancer_df['target'] = target
cancer_df.head()

,mean radius,mean texture,mean perimeter,mean area,mean smoothness,mean compactness,mean concavity,mean concave points,mean symmetry,mean fractal dimension,...,worst texture,worst perimeter,worst area,worst smoothness,worst compactness,worst concavity,worst concave points,worst symmetry,worst fractal dimension,target
0,17.99,10.38,122.80,1001.0,0.11840,0.27760,0.3001,0.14710,0.2419,0.07871,...,17.33,184.60,2019.0,0.1622,0.6656,0.7119,0.2654,0.4601,0.11890,0
1,20.57,17.77,132.90,1326.0,0.08474,0.07864,0.0869,0.07017,0.1812,0.05667,...,23.41,158.80,1956.0,0.1238,0.1866,0.2416,0.1860,0.2750,0.08902,0
2,19.69,21.25,130.00,1203.0,0.10960,0.15990,0.1974,0.12790,0.2069,0.05999,...,25.53,152.50,1709.0,0.1444,0.4245,0.4504,0.2430,0.3613,0.08758,0
3,11.42,20.38,77.58,386.1,0.14250,0.28390,0.2414,0.10520,0.2597,0.09744,...,26.50,98.87,567.7,0.2098,0.8663,0.6869,0.2575,0.6638,0.17300,0
4,20.29,14.34,135.10,1297.0,0.10030,0.13280,0.1980,0.10430,0.1809,0.05883,...,16.67,152.20,1575.0,0.1374,0.2050,0.4000,0.1625,0.2364,0.07678,0


In [86]:
# 양성 357 음성 212명
cancer_df.target.value_counts()

1    357
0    212
Name: target, dtype: int64

In [87]:
X_train,X_test,y_train,y_test = train_test_split(data,target, test_size = 0.8, random_state = 42)

In [88]:
import xgboost as xgb
from xgboost import plot_importance 

In [94]:
model = xgb.XGBClassifier()

In [95]:
model.fit(X_train,y_train)

XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
       colsample_bynode=1, colsample_bytree=1, gamma=0, learning_rate=0.1,
       max_delta_step=0, max_depth=3, min_child_weight=1, missing=None,
       n_estimators=100, n_jobs=1, nthread=None,
       objective='binary:logistic', random_state=0, reg_alpha=0,
       reg_lambda=1, scale_pos_weight=1, seed=None, silent=None,
       subsample=1, verbosity=1)

In [96]:
y_pred = model.predict(X_test)

In [97]:
# 성능 점수
from sklearn.metrics import accuracy_score
accuracy_score(y_test,y_pred)

0.9539473684210527